In [3]:
! pip install web3

In [4]:
## Set up Infura Node

import getpass

# Infura API setup
API_KEY = getpass.getpass()
infura_url = f"https://mainnet.infura.io/v3/{API_KEY}"
web3 = Web3(Web3.HTTPProvider(infura_url))

··········


In [5]:
from web3 import Web3
from decimal import Decimal
from typing import Dict, Tuple, Optional
import json
from datetime import datetime

# ABIs
UNISWAP_PAIR_ABI = json.loads('''
[
    {
        "constant": true,
        "inputs": [],
        "name": "getReserves",
        "outputs": [
            {
                "internalType": "uint112",
                "name": "_reserve0",
                "type": "uint112"
            },
            {
                "internalType": "uint112",
                "name": "_reserve1",
                "type": "uint112"
            },
            {
                "internalType": "uint32",
                "name": "_blockTimestampLast",
                "type": "uint32"
            }
        ],
        "payable": false,
        "stateMutability": "view",
        "type": "function"
    },
    {
        "constant": true,
        "inputs": [],
        "name": "token0",
        "outputs": [
            {
                "internalType": "address",
                "name": "",
                "type": "address"
            }
        ],
        "payable": false,
        "stateMutability": "view",
        "type": "function"
    },
    {
        "constant": true,
        "inputs": [],
        "name": "token1",
        "outputs": [
            {
                "internalType": "address",
                "name": "",
                "type": "address"
            }
        ],
        "payable": false,
        "stateMutability": "view",
        "type": "function"
    }
]
''')

ERC20_ABI = json.loads('''
[
    {
        "constant": true,
        "inputs": [],
        "name": "decimals",
        "outputs": [
            {
                "name": "",
                "type": "uint8"
            }
        ],
        "payable": false,
        "stateMutability": "view",
        "type": "function"
    },
    {
        "constant": true,
        "inputs": [],
        "name": "symbol",
        "outputs": [
            {
                "name": "",
                "type": "string"
            }
        ],
        "payable": false,
        "stateMutability": "view",
        "type": "function"
    }
]
''')

In [6]:
class UniswapLiquidityAnalyzer:
    def __init__(self, web3_provider: str, pair_address: str):
        """
        Initialize the Uniswap liquidity analyzer.

        Args:
            web3_provider: URL of the Web3 provider (e.g., Infura URL)
            pair_address: Address of the Uniswap pair contract
        """
        self.web3 = Web3(Web3.HTTPProvider(web3_provider))
        self.pair_address = Web3.to_checksum_address(pair_address)
        self.pair_contract = self.web3.eth.contract(
            address=self.pair_address,
            abi=UNISWAP_PAIR_ABI
        )
        self.token_info = {}

    async def initialize(self) -> None:
        """Initialize token information for the pair."""
        token0_address = self.pair_contract.functions.token0().call()
        token1_address = self.pair_contract.functions.token1().call()

        token0_contract = self.web3.eth.contract(
            address=token0_address,
            abi=ERC20_ABI
        )
        token1_contract = self.web3.eth.contract(
            address=token1_address,
            abi=ERC20_ABI
        )

        token0_symbol = token0_contract.functions.symbol().call()
        token1_symbol = token1_contract.functions.symbol().call()
        token0_decimals = token0_contract.functions.decimals().call()
        token1_decimals = token1_contract.functions.decimals().call()

        self.token_info = {
            'token0': {
                'address': token0_address,
                'symbol': token0_symbol,
                'decimals': token0_decimals
            },
            'token1': {
                'address': token1_address,
                'symbol': token1_symbol,
                'decimals': token1_decimals
            }
        }

    def get_liquidity_info(self) -> Dict:
        """
        Get current liquidity information for the pair.

        Returns:
            Dictionary containing liquidity information
        """
        reserves = self.pair_contract.functions.getReserves().call()

        reserve0 = Decimal(reserves[0]) / Decimal(10 ** self.token_info['token0']['decimals'])
        reserve1 = Decimal(reserves[1]) / Decimal(10 ** self.token_info['token1']['decimals'])

        return {
            'timestamp': datetime.now().isoformat(),
            'pair': self.pair_address,
            'token0': {
                **self.token_info['token0'],
                'reserve': str(reserve0)
            },
            'token1': {
                **self.token_info['token1'],
                'reserve': str(reserve1)
            },
            'total_liquidity': {
                'token0': f"{reserve0} {self.token_info['token0']['symbol']}",
                'token1': f"{reserve1} {self.token_info['token1']['symbol']}"
            }
        }

    def calculate_usd_value(self, price_feeds: Dict[str, float]) -> Dict:
        """
        Calculate USD value of liquidity using provided price feeds.

        Args:
            price_feeds: Dictionary mapping token symbols to their USD prices

        Returns:
            Dictionary containing liquidity information with USD values
        """
        liquidity_info = self.get_liquidity_info()

        token0_usd = Decimal(liquidity_info['token0']['reserve']) * Decimal(
            price_feeds.get(self.token_info['token0']['symbol'], 0)
        )
        token1_usd = Decimal(liquidity_info['token1']['reserve']) * Decimal(
            price_feeds.get(self.token_info['token1']['symbol'], 0)
        )

        return {
            **liquidity_info,
            'usd_value': {
                'token0': str(token0_usd),
                'token1': str(token1_usd),
                'total': str(token0_usd + token1_usd)
            }
        }

def analyze_pair_liquidity(provider_url: str, pair_address: str) -> None:
    """
    Analyze liquidity for a Uniswap pair.

    Args:
        provider_url: URL of the Web3 provider
        pair_address: Address of the Uniswap pair contract
    """
    try:
        analyzer = UniswapLiquidityAnalyzer(provider_url, pair_address)
        analyzer.initialize()

        liquidity_info = analyzer.get_liquidity_info()
        print('Liquidity Information:')
        print(json.dumps(liquidity_info, indent=2))

        sample_price_feeds = {
            'WETH': 2000,  # Example price
            'USDC': 1
        }
        liquidity_usd = analyzer.calculate_usd_value(sample_price_feeds)
        print('\nLiquidity in USD:')
        print(json.dumps(liquidity_usd, indent=2))

    except Exception as e:
        print(f'Error analyzing liquidity: {str(e)}')

if __name__ == '__main__':
    INFURA_URL = 'YOUR_INFURA_URL'
    PAIR_ADDRESS = 'UNISWAP_PAIR_CONTRACT_ADDRESS'

    analyze_pair_liquidity(INFURA_URL, PAIR_ADDRESS)

Error analyzing liquidity: type object 'Web3' has no attribute 'to_checksum_address'
